In [ ]:
# This file focus on the analyzing the top suicide rate of different countries
# 1. Find out the top 5 countries with highest youth suicide rate 

import os
import math
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append(os.path.abspath('..'))
from common_lib.data_reader import SuicideDataReader, SuicideRawData, SuicideProcessedData

In [ ]:
# define constant variable
ALL_AGE_RANGES = ["all_age", "80_above", "70to79", "60to69", "50to59", "40to49", "30to39", "20to29", "10to19"]
ALL_FACILITIES = ["mental_hospitals", "health_units", "outpatient_facilities", "day_treatment", "residential_facilities"]
COLOR_FOR_SEX = {
    "Male" : "#0000AA",
    "Female" : "#AA0000",
    "Both sexes" : "#00AA00"
}

In [ ]:
# ========= Getting Data =========

data_reader = SuicideDataReader()

suicide_rates_dataframe = data_reader.read_data(SuicideProcessedData.SUICIDE_RATES)
print(suicide_rates_dataframe)

country_facilities_dataframe = data_reader.read_data(SuicideProcessedData.FACILITIES)
print(country_facilities_dataframe)

gini_coef_dataframe = data_reader.read_data(SuicideRawData.SOCIOECONOMIC)
print(gini_coef_dataframe)

In [ ]:
# ========= Prepare Data =========
# join country facilities and gini coef to suicde rate data
suicide_rates_facilities_dataframe = suicide_rates_dataframe.merge(country_facilities_dataframe, how="left", left_on="country", right_on="country")

suicide_rates_facilities_dataframe = suicide_rates_facilities_dataframe.merge(gini_coef_dataframe, how="left", left_on="country", right_on="country")


# distribution of suicide rate of all countries
for facility in ALL_FACILITIES:
    suicide_rates_facilities_dataframe["{0}_cat".format(facility)] = pd.qcut(suicide_rates_facilities_dataframe[facility], 
                                                                     q = 10, 
                                                                     labels = False, 
                                                                     precision = 0)


suicide_rates_facilities_both_sexes_dataframe = suicide_rates_facilities_dataframe[suicide_rates_facilities_dataframe["sex"] == "Both sexes"]

# Sorting and pick the first 5
# top_five_youth_suicide_rates_dataframe = suicide_rates_facilities_dataframe.sort_values(by=["10to19"], ascending=False).head(5)

# # print the results
# top_five_youth_suicide_rates_dataframe

In [ ]:
# ========= plot graph =========
top_suicide_rates_country_dataframe = suicide_rates_facilities_dataframe[suicide_rates_facilities_dataframe["10to19"] > 10].sort_values(by=["10to19"], ascending=False)

#top suicide rate plot
plt.figure(figsize=(20,10))
plt.bar(top_suicide_rates_country_dataframe["country"], top_suicide_rates_country_dataframe["10to19"])
plt.title("Top Youth Suicide Rate Countries \n rate > 10")
plt.xlabel("Country")
plt.ylabel("Suicide Rate (%)")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Check correlation
suicide_rates_facilities_both_sexes_dataframe[ALL_FACILITIES+["gini_coef", "poverty_199", "poverty_320"]].corr()

In [ ]:
# plot the distribution of different facilities
for facility_2 in ALL_FACILITIES:
    plt.figure(figsize=(20,10))
    fig, axs = plt.subplots(nrows=2, ncols=3, constrained_layout=True)
    fig.figsize=(20,10)
    facilities_index = 0	# use for control the location of the plot
    for facility in ALL_FACILITIES:
        x_index = math.floor(facilities_index / 3)
        y_index = facilities_index % 3
        subplot = axs[x_index, y_index]
        subplot.scatter(x = suicide_rates_facilities_dataframe[suicide_rates_facilities_dataframe["sex"] == "Both sexes"]["{0}".format(facility_2)], 
                        y = suicide_rates_facilities_dataframe[suicide_rates_facilities_dataframe["sex"] == "Both sexes"]["{0}".format(facility)], 
                        color = "#00AA00", 
                        s = 5)
        subplot.set_title("{0}".format(facility))
        
        plt.suptitle("{0} facilities".format(facility_2))
        facilities_index = facilities_index + 1

    plt.show()

In [ ]:
# plot the suicide rate vs different facilities for analysis
ALL_SEX = top_suicide_rates_country_dataframe["sex"].unique()



for sex in ALL_SEX:
    sex_color = COLOR_FOR_SEX[sex]
    for age_range in ALL_AGE_RANGES:
        plt.figure(figsize=(20,10))
        fig, axs = plt.subplots(nrows=2, ncols=3, constrained_layout=True)
        fig.figsize=(20,10)
        facilities_index = 0	# use for control the location of the plot
        for facility in ALL_FACILITIES:
            x_index = math.floor(facilities_index / 3)
            y_index = facilities_index % 3
            subplot = axs[x_index, y_index]
            subplot.scatter(x = suicide_rates_facilities_both_sexes_dataframe[age_range], 
                            y = suicide_rates_facilities_both_sexes_dataframe[facility], 
                            color = sex_color, 
                            s = 5)
            subplot.set_title("{0}".format(facility))
            #subplot.xlabel("{0} {1} suicide Rate (%)".format(sex, age_range))
            #subplot.ylabel("{0}".format(facility))
            plt.suptitle("{0} {1} vs different facilities".format(sex, age_range))
            facilities_index = facilities_index + 1

        plt.show()

In [ ]:
for sex in ALL_SEX:
    sex_color = COLOR_FOR_SEX[sex]
    for age_range in ALL_AGE_RANGES:
        plt.figure(figsize=(20,10))
        fig, axs = plt.subplots(nrows=2, ncols=3, constrained_layout=True)
        fig.figsize=(20,10)
        facilities_index = 0	# use for control the location of the plot
        for facility in ALL_FACILITIES:
            x_index = math.floor(facilities_index / 3)
            y_index = facilities_index % 3
            subplot = axs[x_index, y_index]
            subplot.scatter(x = suicide_rates_facilities_dataframe[suicide_rates_facilities_dataframe["sex"] == sex][age_range], 
                            y = suicide_rates_facilities_dataframe[suicide_rates_facilities_dataframe["sex"] == sex]["{0}_cat".format(facility)], 
                            color = sex_color, 
                            s = 5)
            subplot.set_title("{0}".format(facility))
            #subplot.xlabel("{0} {1} suicide Rate (%)".format(sex, age_range))
            #subplot.ylabel("{0}".format(facility))
            plt.suptitle("{0} {1} vs different facilities".format(sex, age_range))
            facilities_index = facilities_index + 1

        plt.show()

In [ ]:
# check soc-econ vs facilities
test_variable = "gini_coef"
test_variable = "poverty_199"
plt.figure(figsize=(20,10))
fig, axs = plt.subplots(nrows=2, ncols=3, constrained_layout=True)
fig.figsize=(20,10)
facilities_index = 0	# use for control the location of the plot
for facility in ALL_FACILITIES:
    x_index = math.floor(facilities_index / 3)
    y_index = facilities_index % 3
    subplot = axs[x_index, y_index]
    
    subplot.scatter(x = suicide_rates_facilities_both_sexes_dataframe[test_variable], 
                y = suicide_rates_facilities_both_sexes_dataframe["{0}".format(facility)], 
                color = "#00AA00", 
                s = 5)
    #plt.xlabel(test_variable)
    #plt.ylabel("facility")
    subplot.set_title("{0}".format(facility))
    #plt.title("{0} vs gini coef".format(facility))
    facilities_index = facilities_index + 1
plt.show()

In [ ]:
# facilites distribution
plt.figure(figsize=(40,20))
fig, axs = plt.subplots(nrows=2, ncols=3, constrained_layout=True)
fig.figsize=(40,20)
facilities_index = 0	# use for control the location of the plot
for facility in ALL_FACILITIES:
    x_index = math.floor(facilities_index / 3)
    y_index = facilities_index % 3
    subplot = axs[x_index, y_index]
    
    subplot.hist(x = suicide_rates_facilities_both_sexes_dataframe["{0}".format(facility)], 
                 bins = 25,
                color = "#00AA00")
    #plt.xlabel(test_variable)
    #plt.ylabel("facility")
    subplot.set_title("{0}".format(facility))
    #plt.title("{0} vs gini coef".format(facility))
    facilities_index = facilities_index + 1
plt.show()